In [1]:
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
model = Sequential()
# Select 6 feature convolution kernels with a size of 3 * 3 (without offset), and get 6 feature maps. The size of each feature map is 32−3 + 1 = 30.
# That is, the number of neurons has been reduced from 1024 to 30 ∗ 30 = 900 .
# Parameters between input layer and C1 layer: 6 ∗ (3 ∗ 3 + 1)
model.add(Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
# The input of this layer is the output of the first layer, which is a 30 * 30 * 6 node matrix.
# The size of the filter used in this layer is 2 * 2, and the step length and width are both 2, so the output matrix size of this layer is 14 * 14 * 6.
model.add(MaxPooling2D(pool_size=(2, 2)))
# The input matrix size of this layer is 15 * 15 * 6, the filter size used is 3 * 3, and the depth is 16. This layer does not use all 0 padding, and the step size is 1.
# The output matrix size of this layer is 13 * 13 * 16. This layer has 3 * 3 * 6 * 16 + 16 = 880 parameters
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
# The input matrix size of this layer is 13 * 13- * 16. The size of the filter used in this layer is 2 * 2, and the length and width steps are both 2, so the output matrix size of this layer is 5 * 5 * 16.
model.add(MaxPooling2D(pool_size=(2, 2)))
# The input matrix size of this layer is 6 * 6 * 16. This layer is called a convolution layer in the LeNet-5 paper, but because the size of the filter is 3 * 3, #
# So it is not different from the fully connected layer. If the nodes in the 3 * 3 * 16 matrix are pulled into a vector, then this layer is the same as the fully connected layer.
# The number of output nodes in this layer is 120, with a total of 3 * 3 * 16 * 120 + 120 = 17400 parameters.
model.add(Flatten())
model.add(Dense(120, activation='relu'))
# The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
model.add(Dense(84, activation='relu'))
# The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
model.add(Dense(5, activation='softmax'))
model.compile(loss=keras.metrics.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [3]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('master_dataset/train',
                                                 target_size = (32, 32),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('master_dataset/test',
                                            target_size = (32, 32),
                                            batch_size = 16,
                                            class_mode = 'categorical')
model.fit_generator(training_set,
                    steps_per_epoch = len(training_set),
                    epochs = 25,
                    validation_data = test_set,    
                    validation_steps = len(test_set))

Found 47 images belonging to 5 classes.
Found 10 images belonging to 5 classes.
Epoch 1/25
3/3 [==============================] - 1s 182ms/step - loss: 1.6386 - accuracy: 0.2340 - val_loss: 1.6042 - val_accuracy: 0.3000
Epoch 2/25
3/3 [==============================] - 0s 101ms/step - loss: 1.5636 - accuracy: 0.3404 - val_loss: 1.5826 - val_accuracy: 0.3000
Epoch 3/25
3/3 [==============================] - 0s 124ms/step - loss: 1.5410 - accuracy: 0.3617 - val_loss: 1.5679 - val_accuracy: 0.3000
Epoch 4/25
3/3 [==============================] - 0s 107ms/step - loss: 1.5015 - accuracy: 0.3830 - val_loss: 1.5405 - val_accuracy: 0.5000
Epoch 5/25
3/3 [==============================] - 0s 131ms/step - loss: 1.4656 - accuracy: 0.4255 - val_loss: 1.5108 - val_accuracy: 0.4000
Epoch 6/25
3/3 [==============================] - 0s 145ms/step - loss: 1.4126 - accuracy: 0.4043 - val_loss: 1.4661 - val_accuracy: 0.5000
Epoch 7/25
3/3 [==============================] - 0s 121ms/step - loss: 1.3602 -

In [4]:
model.save("model_Leenet.h5")
print("Saved model to disk")


Saved model to disk


In [8]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('master_dataset/test/Spider/spider 13.jpg', target_size = (32, 32))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices
print(result)


[[0. 0. 0. 0. 1.]]


In [9]:
if result[0][0] == 1:
    prediction = 'Apple'
    print(prediction)
elif result[0][1] == 2:
    prediction = 'Cat'
    print(prediction)
elif result[0][2] == 3:
    prediction = 'Dog'
    print(prediction)
elif result[0][3] == 4:
    prediction = 'Lemon'
    print(prediction)
else:
    prediction = 'Spider'
    print(prediction)

Spider
